In [ ]:
import re
import numpy as np
import pandas as pd 
import missingno as msno
from text_similarity import *

from nltk.corpus import stopwords
portugues_stops = stopwords.words('portuguese')
portugues_stops = [i.upper() for i in portugues_stops] 

from sklearn.preprocessing import Binarizer

# Importar os dados

In [ ]:
df = pd.read_csv('data/dados-pacote-cirurgico-2022-02-08 10_30_25.csv', sep=';', encoding='iso8859-1')

-  Remover espaços desnecessarios e '*'

In [ ]:
size = df['Descrição do Item'].shape[0]

df['Descrição do Item'] = [df['Descrição do Item'][i].replace('*', '').strip() for i in range(size)]
df['Descrição do Item'] = [re.sub(' +',' ', df['Descrição do Item'][i]) for i in range(size)]

- Pegar itens unicos

In [ ]:
descricao_itens = df['Descrição do Item'].unique()

- Remover stopwords

In [ ]:
itens = pd.Series(descricao_itens).apply(lambda item: remove_stopwords(item, portugues_stops))

# Matriz de similaridade

- Gernado a Matriz de similaridade de Jaro

In [ ]:
matriz_jaro_winkler = similarity_matrix(np.array(itens), metric='jaro_winkler')

- Convertendo matriz de similaridade para Dataframe

In [ ]:
matriz_similaridade = pd.DataFrame(matriz_jaro_winkler, index=descricao_itens, columns=descricao_itens) 

In [ ]:
matriz_similaridade

- Converter Matriz para dataframe

In [ ]:
df_similaridade = matriz_similaridade.unstack().reset_index().dropna()

df_similaridade.rename(columns = {'level_0': 'item_A', 'level_1': 'item_B', 0:'valor_similaridade'}, inplace = True)

df_similaridade

## Binarizer

In [ ]:
df_similaridade['similares'] = Binarizer(threshold=0.8).fit_transform(df_similaridade[['valor_similaridade']])

In [ ]:
bins = [-1, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

labels = [
    "00 -| 10", "11 -| 20", "21 -| 30", "31 -| 40", "41 -| 50", 
    "51 -| 60", "61 -| 70", "71 -| 80", "81 -| 90", "91 -| 100"
]

df_similaridade['classe'] = creat_class(df_similaridade['valor_similaridade'],bins, labels)

In [ ]:
df_similaridade

# Explorando os Resultados

In [ ]:
df_similaridade = df_similaridade.query('valor_similaridade > 0 and valor_similaridade < 1.0')

### Analisar por Grupos 

In [ ]:
list_cateter_cetrofix = [
    'CATETER CERTOFIX DUO PAED S420 G22,22 20CM 4166949',
    'CATETER CERTOFIX MONO PAED S110 G22 10CM 4160177'
]

list_cateter_venoso = [
    'CATETER VENOSO PERIFERICO 22G - AUTOGUARD 381823 BD',
    'CATETER VENOSO PERIFERICO 24G - AUTOGUARD 381812 BD'
    'CATETER INTRAVASCULAR PERIFERICO JELCO 22GX25MM AZUL - SMITH'
]

list_fio_monolion = [
    'FIO CIRURGICO MONONYLON P1662 T 4-0 - JOHNSON & JOHNSON',
    'FIO CIRURGICO MONONYLON P1664 T 4-0 - JOHNSON & JOHNSON',
    'FIO CIRURGICO MONONYLON P1663 T 3-0 - JOHNSON E JOHNSON',
    'FIO CIRURGICO MONONYLON 1215 T 2-0 - JOHNSON & JOHNSON'
]

list_fio_prolene = [
    'FIO CIRURGICO PROLENE 8707 T 6-0 - JOHNSON & JOHNSON',
    'FIO CIRURGICO PROLENE 9556 T 5-0 - JOHNSON & JOHNSON',
    'FIO CIRURGICO PROLENE M8703 T 7-0 - JOHNSON & JOHNSON',
    'FIO CIRURGICO PROLENE 8704 T 7-0 - JOHNSON & JOHNSON',
    'FIO CIRURGICO PROLENE 8730 T 8-0'
]

list_fio_seda = [
    'FIO CIRURGICO SEDA G813 T 2-0 - JOHNSON & JOHNSON',
    'FIO CIRURGICO SEDA SSP15 T-0'
    'FIO CIRURGICO SEDA SSP13 T 3-0 - JOHNSON & JOHNSON'
    'FIO CIRURGICO SEDA SSP14 T 2-0 - JOHNSON E JOHNSON'
]

list_fio_vicryl = [
    'FIO CIRURGICO VICRYL VCP316 H - JOHNSON & JOHNSON',
    'FIO CIRURGICO VICRYL VCP340 H 0 - JOHNSON & JOHNSON'
]

list_bisturi = [
    'LAMINA DE BISTURI NR 11 ESTERIL - B. BRAUN',
    'LAMINA DE BISTURI NR 15 ESTERIL - B. BRAUN',
    'LAMINA DE BISTURI NR 21 ESTERIL - B. BRAUN'
]

list_perfursor =[
    'PERFUSOR SET 120CM - B. BRAUN',
    'PERFUSOR SET 20CM - B. BRAUN'
]

list_cauterio = [
    'PLACA DE CAUTERIO INFANTIL SEM CABO REF: 1146 - 3M', 
    'PLACA ELETROCIRURGICA PARA PACIENTE AUTOADESIVA RN'
]

list_frauda = [
    'FRALDA NEONATAL TAM. PP (0 A 3,5KG)',
    'FRALDA INFANTIL P (ATE 6KG)',
    'FRALDA INFANTIL EG (ACIMA DE 12KG)',
    'FRALDA INFANTIL M (DE 6 A 10KG)',
    'FRALDA PROTEK XXG 14 A 18 KG'
]

list_sonda = [
    'SONDA FOLEY 2 VIAS 06 SILICONE - RUSCH',
    'SONDA FOLEY 2 VIAS 08 BALAO 30 - RUSCH',
    'SONDA FOLEY 2 VIAS 08 SILICONE - RUSCH'
]

list_tubo_endotraqueal = [
    'TUBO ENDOTRAQUEAL S/BALAO 3.0 - MALLINCKRODT',
    'TUBO ENDOTRAQUEAL S/BALAO 4.0 - MALLINCKRODT',
    'TUBO ENDOTRAQUEAL S/BALAO 3.5 - MALLINCKRODT',
    'TUBO ENDOTRAQUEAL C/BALAO 4.0 - MALLINCKRODT'
]

list_sistema_drenagem = [
    'SISTEMA DRENAGEM MEDIASTINAL Nº 10 250ML',
    'SISTEMA DRENAGEM MEDIASTINAL Nº 12 250ML',
    'SISTEMA DRENAGEM MEDIASTINAL Nº 16 2000ML P.SIMON',
    'SISTEMA DRENAGEM MEDIASTINAL Nº 18 2000ML P.SIMON'
]

## Filtro cateter cetrofix

- Filtros de 90% e 80%

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(list_cateter_cetrofix)].query('valor_similaridade >= 0.9')

In [ ]:
df_cateter_cetrofix = df_similaridade[df_similaridade['item_B'].isin(list_cateter_cetrofix)].query('valor_similaridade >= 0.8')

In [ ]:
df_cateter_cetrofix

- Agrupando itens das colunas itens_A e itens_B

In [ ]:
list_cateter_cetrofix

In [ ]:
group_cateter_cetrofix = group_similarity(df_cateter_cetrofix, 'item_A', 'item_B')

In [ ]:
group_cateter_cetrofix

- Refinando o modelo com regex

In [ ]:
regex = re.compile(r'\b(\w+\s+\w+)\b')
cateter_cetrofix = regex_similarity('CATETER CERTOFIX', regex, group_cateter_cetrofix['itens'])

In [ ]:
cateter_cetrofix

In [ ]:
group_cateter_cetrofix = group_cateter_cetrofix[group_cateter_cetrofix['itens'].isin(cateter_cetrofix)]
group_cateter_cetrofix

In [ ]:
group_cateter_cetrofix.to_excel('data/refined_group/cateter_cetrofix.xlsx', index=False)

## Filtro cateter venoso

- Filtros de 90% e 80%

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(list_cateter_venoso)].query('valor_similaridade >= 0.9')

In [ ]:
df_cateter_venoso = df_similaridade[df_similaridade['item_B'].isin(list_cateter_venoso)].query('valor_similaridade >= 0.8')

In [ ]:
df_cateter_venoso

- Agrupando itens das colunas itens_A e itens_B

In [ ]:
group_cateter_venoso = group_similarity(df_cateter_venoso, 'item_A', 'item_B')
group_cateter_venoso

- Refinando o modelo com regex

In [ ]:
regex = re.compile(r'\b(\w+\s+\w+)\b')
cateter_venoso = regex_similarity('CATETER VENOSO', regex, group_cateter_venoso['itens'])

In [ ]:
cateter_venoso

In [ ]:
group_cateter_venoso = group_cateter_venoso[group_cateter_venoso['itens'].isin(cateter_venoso)]
group_cateter_venoso

In [ ]:
group_cateter_venoso.to_excel('data/refined_group/cateter_venoso.xlsx', index=False)

## Filtro fio Prolene

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(list_fio_prolene)].query('valor_similaridade >= 0.9')

In [ ]:
df_fio_prolene = df_similaridade[df_similaridade['item_B'].isin(list_fio_prolene)].query('valor_similaridade >= 0.8')

- Agrupando itens das colunas itens_A e itens_B

In [ ]:
group_fio_prolene = group_similarity(df_fio_prolene, 'item_A', 'item_B')
group_fio_prolene

- Refinando o modelo com regex

In [ ]:
regex = re.compile(r'\b(\w+\s+\w+\s+\w+)\b')

fio_prolene = regex_similarity('FIO CIRURGICO PROLENE', regex, group_fio_prolene['itens'])

In [ ]:
fio_prolene

In [ ]:
group_fio_prolene = group_fio_prolene[group_fio_prolene['itens'].isin(fio_prolene)]
group_fio_prolene

In [ ]:
group_fio_prolene.shape

In [ ]:
group_fio_prolene.to_excel('data/refined_group/fio_cirurugico_prolene.xlsx', index=False)

## Filtro Fio seda

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(list_fio_seda)].query('valor_similaridade >= 0.9')

In [ ]:
df_fio_seda = df_similaridade[df_similaridade['item_B'].isin(list_fio_seda)].query('valor_similaridade >= 0.8')
df_fio_seda

- Agrupando itens das colunas itens_A e itens_B

In [ ]:
group_fio_seda = group_similarity(df_fio_seda, 'item_A', 'item_B')
group_fio_seda

- Refinando o modelo com regex

In [ ]:
regex = re.compile(r'\b(\w+\s+\w+\s+\w+)\b')

fio_seda = regex_similarity('FIO CIRURGICO SEDA', regex, group_fio_seda['itens'])

In [ ]:
group_fio_prolene = group_fio_prolene[group_fio_prolene['itens'].isin(fio_prolene)]
group_fio_prolene

In [ ]:
group_fio_seda = group_fio_seda[group_fio_seda['itens'].isin(fio_seda)]
group_fio_seda

In [ ]:
group_fio_seda.to_excel('data/refined_group/fio_cirurugico_seda.xlsx', index=False)

## Filtro Fio Vicryl

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(list_fio_vicryl)].query('valor_similaridade >= 0.9')

In [ ]:
df_fio_vicryl = df_similaridade[df_similaridade['item_B'].isin(list_fio_vicryl)].query('valor_similaridade >= 0.8')

In [ ]:
df_fio_vicryl

- Agrupando itens das colunas itens_A e itens_B

In [ ]:
group_fio_vicryl = group_similarity(df_fio_vicryl, 'item_A', 'item_B')
group_fio_vicryl

- Refinando o modelo com regex

In [ ]:
regex = re.compile(r'\b(\w+\s+\w+\s+\w+)\b')

fio_vicry = regex_similarity('FIO CIRURGICO VICRYL', regex, group_fio_vicryl['itens'])

In [ ]:
group_fio_vicryl = group_fio_vicryl[group_fio_vicryl['itens'].isin(fio_vicry)]

In [ ]:
group_fio_vicryl.to_excel('data/refined_group/fio_cirurugico_vicryl.xlsx', index=False)

## Filtro bisturi

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(list_bisturi)].query('valor_similaridade >= 0.9')

In [ ]:
df_bisturi = df_similaridade[df_similaridade['item_B'].isin(list_bisturi)].query('valor_similaridade >= 0.8')

- Agrupando itens das colunas itens_A e itens_B

In [ ]:
group_bisturi = group_similarity(df_bisturi, 'item_A', 'item_B')
group_bisturi

## Filtro Perfursor

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(list_perfursor)].query('valor_similaridade >= 0.9')

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(list_perfursor)].query('valor_similaridade >= 0.8')

- Agrupando itens das colunas itens_A e itens_B

In [ ]:
group_bisturi = group_similarity(df_perfursor, 'item_A', 'item_B')
group_bisturi

## Filtro tubo endotraqueal

In [ ]:
tb_endotraqueal_09 = df_similaridade[df_similaridade['item_B'].isin(list_tubo_endotraqueal)].query('valor_similaridade >= 0.9')

In [ ]:
group_endotraqueal_09 = group_similarity(tb_endotraqueal_09, 'item_A', 'item_B')

In [ ]:
group_endotraqueal_09.shape

In [ ]:
df_endotraqueal = df_similaridade[df_similaridade['item_B'].isin(list_tubo_endotraqueal)].query('valor_similaridade >= 0.8')

- Agrupando itens das colunas itens_A e itens_B

In [ ]:
group_endotraqueal = group_similarity(df_endotraqueal, 'item_A', 'item_B')
group_endotraqueal

In [ ]:
group_endotraqueal.shape

- Refinando o modelo com regex

In [78]:
regex = re.compile(r'\b(\w+\s+\w+)\b')

tubo_endotraqueal = regex_similarity('TUBO ENDOTRAQUEAL', regex, group_endotraqueal['itens'])

In [80]:
group_endotraqueal = group_endotraqueal[group_endotraqueal['itens'].isin(tubo_endotraqueal)]

In [81]:
group_endotraqueal

,itens,predito
1490649,TUBO ENDOTRAQUEAL C/BALAO 4.0 - MALLINCKRODT,1
1720905,TUBO ENDOTRAQUEAL TAPERGUARD C/BALAO 7.5 - MAL...,1
1942305,TUBO ENDOTRAQUEAL TAPERGUARD C/BALAO 5.5 - MAL...,1
2112783,TUBO ENDOTRAQUEAL TAPERGUARD C/BALAO 6.0 - MAL...,1
2506875,TUBO ENDOTRAQUEAL C/BALAO 3.5 - MALLINCKRODT,1
2579937,TUBO ENDOTRAQUEAL C/BALAO 4.5 - MALLINCKRODT,1
2763699,TUBO ENDOTRAQUEAL TAPERGUARD C/BALAO 5.0 - MAL...,1
2785839,TUBO ENDOTRAQUEAL TAPERGUARD C/BALAO 9.0 - MAL...,1
2934177,TUBO ENDOTRAQUEAL ARAMADO C/BALAO - 7.0 - MALL...,1
3086943,TUBO ENDOTRAQUEAL S/BALAO 3.0 - MALLINCKRODT,1


In [86]:
group_endotraqueal.to_excel('data/refined_group/tubo_endotraqueal.xlsx', index=False)

## Filtrando fio monolion

In [87]:
df_fio_monolion_09 = df_similaridade[df_similaridade['item_B'].isin(list_fio_monolion)].query('valor_similaridade >= 0.9')

In [88]:
group_fio_mono_09 = group_similarity(df_fio_monolion_09, 'item_A', 'item_B')
group_fio_mono_09.shape

(9, 2)

In [89]:
group_fio_mono_09

,itens,predito
524941,FIO CIRURGICO MONONYLON P1662 T 4-0 - JOHNSON ...,1
706489,FIO CIRURGICO MONONYLON P1664 T 4-0 - JOHNSON ...,1
803905,FIO CIRURGICO MONONYLON 1163 T 3-0,1
1492459,FIO CIRURGICO MONONYLON 1215 T 2-0 - JOHNSON &...,1
3487273,FIO CIRURGICO MONONYLON P1602 T 5-0 - JOHNSON ...,1
3489487,FIO CIRURGICO MONONYLON P1611 T 4-0 - JOHNSON ...,1
3498343,FIO CIRURGICO MONONYLON P1661 T 5-0 - JOHNSON ...,1
3912361,FIO CIRURGICO MONONYLON P1666 T 5-0 - JOHNSON ...,1
524941,FIO CIRURGICO MONONYLON P1663 T 3-0 - JOHNSON ...,1


In [90]:
df_fio_monolion = df_similaridade[df_similaridade['item_B'].isin(list_fio_monolion)].query('valor_similaridade >= 0.8')

In [91]:
df_fio_monolion

,item_A,item_B,valor_similaridade,similares,classe
524941,FIO CIRURGICO MONONYLON P1662 T 4-0 - JOHNSON ...,FIO CIRURGICO MONONYLON P1663 T 3-0 - JOHNSON ...,0.984906,1.0,91 -| 100
706489,FIO CIRURGICO MONONYLON P1664 T 4-0 - JOHNSON ...,FIO CIRURGICO MONONYLON P1663 T 3-0 - JOHNSON ...,0.984906,1.0,91 -| 100
706503,FIO CIRURGICO MONONYLON P1664 T 4-0 - JOHNSON ...,FIO CIRURGICO MONONYLON P1662 T 4-0 - JOHNSON ...,0.984761,1.0,91 -| 100
761839,FIO CIRURGICO CATGUT SIMPLES 853 T 2-0 - JOHNS...,FIO CIRURGICO MONONYLON P1663 T 3-0 - JOHNSON ...,0.846586,1.0,81 -| 90
761853,FIO CIRURGICO CATGUT SIMPLES 853 T 2-0 - JOHNS...,FIO CIRURGICO MONONYLON P1662 T 4-0 - JOHNSON ...,0.846586,1.0,81 -| 90
...,...,...,...,...,...
4569919,FIO CIRURGICO PDS II Z339 2-0- JOHNSON & JOHNSON,FIO CIRURGICO MONONYLON P1663 T 3-0 - JOHNSON ...,0.855197,1.0,81 -| 90
4569933,FIO CIRURGICO PDS II Z339 2-0- JOHNSON & JOHNSON,FIO CIRURGICO MONONYLON P1662 T 4-0 - JOHNSON ...,0.845456,1.0,81 -| 90
4570015,FIO CIRURGICO PDS II Z339 2-0- JOHNSON & JOHNSON,FIO CIRURGICO MONONYLON P1664 T 4-0 - JOHNSON ...,0.841033,1.0,81 -| 90
4570370,FIO CIRURGICO PDS II Z339 2-0- JOHNSON & JOHNSON,FIO CIRURGICO MONONYLON 1215 T 2-0 - JOHNSON &...,0.843718,1.0,81 -| 90


- Agrupando itens das colunas itens_A e itens_B

In [92]:
group_fio_mono = group_similarity(df_fio_monolion, 'item_A', 'item_B')
group_fio_mono

,itens,predito
524941,FIO CIRURGICO MONONYLON P1662 T 4-0 - JOHNSON ...,1
706489,FIO CIRURGICO MONONYLON P1664 T 4-0 - JOHNSON ...,1
761839,FIO CIRURGICO CATGUT SIMPLES 853 T 2-0 - JOHNS...,1
803905,FIO CIRURGICO MONONYLON 1163 T 3-0,1
845971,FIO CIRURGICO MONOCRYL Y495 H5-0 - JOHNSON E J...,1
848185,FIO CIRURGICO VICRYL VCP304 H 4-0 - JOHNSON & ...,1
914605,FIO CIRURGICO SEDA G813 T 2-0 - JOHNSON & JOHNSON,1
919033,FIO CIRURGICO VICRYL VCP333 H 2-0 - JOHNSON & ...,1
950029,FIO CIRURGICO PROLENE P8682 T 4-0 - JOHNSON & ...,1
961099,FIO CIRURGICO VICRYL VCP351 H 2-0 - JOHNSON & ...,1


In [93]:
group_fio_mono.shape

(60, 2)

- Refinando o modelo com regex

In [94]:
regex = re.compile(r'\b(\w+\s+\w+\s+\w+)\b')

fio_monolion = regex_similarity('FIO CIRURGICO MONONYLON', regex, group_fio_mono['itens'])

In [95]:
group_fio_mono = group_fio_mono[group_fio_mono['itens'].isin(fio_monolion)]

In [96]:
group_fio_mono.shape

(10, 2)

In [97]:
group_fio_mono.to_excel('data/refined_group/fio_mono.xlsx')

## Filtrando frauda

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(list_frauda)].query('valor_similaridade >= 0.9')

In [ ]:
group_frauda = df_similaridade[df_similaridade['item_B'].isin(list_frauda)].query('valor_similaridade >= 0.8')

- Agrupando itens das colunas itens_A e itens_B

In [ ]:
group_frauda = group_similarity(group_frauda, 'item_A', 'item_B')
group_frauda

## Filtro Sonda

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(list_sonda)].query('valor_similaridade >= 0.9')

In [ ]:
group_sonda_foley = df_similaridade[df_similaridade['item_B'].isin(list_sonda)].query('valor_similaridade >= 0.8')

- Agrupando itens das colunas itens_A e itens_B

In [ ]:
group_sonda_foley = group_similarity(group_sonda_foley, 'item_A', 'item_B')
group_sonda_foley